In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

C:\Users\YA MAALIKUL MULK\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [21]:
movies = pd.read_csv("C:/Users/YA MAALIKUL MULK/Downloads/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.shape

(62423, 3)

In [5]:
rating=pd.read_csv("C:/Users/YA MAALIKUL MULK/Downloads/ratings (1).csv")
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
rating.shape

(25000095, 4)

In [7]:
tag=pd.read_csv("C:/Users/YA MAALIKUL MULK/Downloads/tags.csv")
tag.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [8]:
tag.shape

(1093360, 4)

In [9]:
movies['year']=movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II (1995),Comedy,(1995)
...,...,...,...,...
62418,209157,We (2018),Drama,(2018)
62419,209159,Window of the Soul (2001),Documentary,(2001)
62420,209163,Bad Poems (2018),Comedy|Drama,(2018)
62421,209169,A Girl Thing (2001),(no genres listed),(2001)


In [10]:
movies['year']=movies.year.str.extract('(\d\d\d\d)',expand=False)
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
62418,209157,We (2018),Drama,2018
62419,209159,Window of the Soul (2001),Documentary,2001
62420,209163,Bad Poems (2018),Comedy|Drama,2018
62421,209169,A Girl Thing (2001),(no genres listed),2001


In [11]:
movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')
movies.head()

C:\Users\YA MAALIKUL MULK\AppData\Local\Temp\ipykernel_29964\1356428212.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [12]:
movies['title']=movies.title.str.strip()
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [13]:
rating.groupby('movieId').rating.mean
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [14]:
df=movies.join(rating,lsuffix='N',rsuffix='K')  
df

,movieIdN,title,genres,year,userId,movieIdK,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,296,5.0,1147880044
1,2,Jumanji,Adventure|Children|Fantasy,1995,1,306,3.5,1147868817
2,3,Grumpier Old Men,Comedy|Romance,1995,1,307,5.0,1147868828
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,1,665,5.0,1147878820
4,5,Father of the Bride Part II,Comedy,1995,1,899,3.5,1147868510
...,...,...,...,...,...,...,...,...
62418,209157,We,Drama,2018,494,102903,3.5,1532809067
62419,209159,Window of the Soul,Documentary,2001,494,103042,3.0,1532809256
62420,209163,Bad Poems,Comedy|Drama,2018,494,103249,3.0,1532809116
62421,209169,A Girl Thing,(no genres listed),2001,494,103372,3.0,1532911673


In [15]:
df=df.drop(['movieIdK','year','genres','timestamp'],axis=1)
df

,movieIdN,title,userId,rating
0,1,Toy Story,1,5.0
1,2,Jumanji,1,3.5
2,3,Grumpier Old Men,1,5.0
3,4,Waiting to Exhale,1,5.0
4,5,Father of the Bride Part II,1,3.5
...,...,...,...,...
62418,209157,We,494,3.5
62419,209159,Window of the Soul,494,3.0
62420,209163,Bad Poems,494,3.0
62421,209169,A Girl Thing,494,3.0


In [16]:
from scipy.sparse import csr_matrix


In [22]:
movie_users = df.pivot(index='movieIdN',columns='userId',values='rating').fillna(0)
matrix_movies_users=csr_matrix(movie_users.values)

In [23]:
matrix_movies_users

<62423x494 sparse matrix of type '<class 'numpy.float64'>'
	with 62423 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.neighbors import NearestNeighbors


In [28]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
knn.fit(matrix_movies_users)


NearestNeighbors(algorithm='brute', metric='cosine')

In [29]:
knn.fit(matrix_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine')

In [34]:
def recommender(movie_name, data, model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df['title'])[2]
    print('Movie Selected:-',df['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df['title'][i].where(i!=idx))







In [35]:
recommender('Toy Story', matrix_movies_users, knn, 5)


Movie Selected:- Toy Story Index:  0
Searching for recommendations.....
64                         Bio-Dome
49              Usual Suspects, The
47                       Pocahontas
45    How to Make an American Quilt
50                   Guardian Angel
Name: title, dtype: object


In [36]:
recommender('Sudden Death', matrix_movies_users, knn, 5)


Movie Selected:- Sudden Death Index:  8
Searching for recommendations.....
64                         Bio-Dome
49              Usual Suspects, The
47                       Pocahontas
45    How to Make an American Quilt
50                   Guardian Angel
Name: title, dtype: object


In [37]:
recommender('Bad Poems', matrix_movies_users, knn, 5)


Movie Selected:- Bad Poems Index:  62420
Searching for recommendations.....
62064                                             Netizens
62065                                         True Fiction
62063    Takumi - A 60,000 hour story on the survival o...
62061                                            Seventeen
62066                   A Cinderella Story: Christmas Wish
Name: title, dtype: object


In [38]:
recommender('Jumanji', matrix_movies_users, knn, 5)


Movie Selected:- Jumanji Index:  1
Searching for recommendations.....
64                         Bio-Dome
49              Usual Suspects, The
47                       Pocahontas
45    How to Make an American Quilt
50                   Guardian Angel
Name: title, dtype: object
